# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import os
import gmaps
import gmaps.datasets
# Import API key
from api_keys import (gkey, weather_api_key)
import pycountry

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#load weather data from file
city_weather_df = pd.read_csv("../output_data/City_Weather_Data.csv", dtype="object", encoding = "utf-8")
#clean the data
city_weather_df.dropna()
city_weather_df = city_weather_df.drop(columns=["Unnamed: 0"])
city_weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,ribeira grande,38.52,-28.7,69.19,76,72,3.53,PT,1595203321
1,red wing,44.56,-92.53,82.4,34,1,12.75,US,1595203321
2,mar del plata,-38.0,-57.56,52.0,93,75,12.75,AR,1595203321
3,sangar,63.92,127.47,73.56,58,94,12.24,RU,1595203322
4,taywarah,33.35,64.42,51.53,45,0,6.29,AF,1595203322


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#config the api key
gmaps.configure(api_key=gkey)
#setup coodinate
locations = city_weather_df[['Lat', 'Lng']].astype(float)
#Humidity as the weight.
humidity_rate = city_weather_df["Humidity"].astype(float)
#setup the center of map
fig = gmaps.figure(center=(33.525665, -117.626664), zoom_level=2)
gmap_layer = gmaps.heatmap_layer(
                                locations, 
                                weights = humidity_rate,
                                dissipating=False,
                                max_intensity=100,
                                point_radius = 3)
# Add layer
fig.add_layer(gmap_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#create dataframe for display

ideal_weather_condition = city_weather_df.loc[(city_weather_df.loc[:,"Max Temp"].astype(float)>=70) 
                                              # Filter vacation with max temp above 70 degrees F
                                & (city_weather_df.loc[:,"Max Temp"].astype(float)<=80)
                                              # Filter vacation with max temp below 80 degrees F
                                & (city_weather_df.loc[:,"Wind Speed"].astype(float)<=10)
                                              # Filter vacation with wind speed below 10 mph
                                & (city_weather_df.loc[:,"Cloudiness"].astype(float)==0)
                                              # Filter vacation with zero cloudiness
                               ]
#Drop any rows will null values
ideal_weather_condition.dropna()
ideal_weather_condition.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
161,jining,35.41,116.58,78.17,59,0,3.22,CN,1595203352
165,yongan,39.7,113.69,72.75,42,0,4.74,CN,1595203353
188,cherskiy,68.75,161.3,72.34,40,0,4.81,RU,1595203176
240,gonbad-e qabus,37.25,55.17,77.0,73,0,2.24,IR,1595203327
278,plomarion,38.98,26.37,73.4,78,0,4.7,GR,1595203376


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# reset Index
#Store into variable named `hotel_df`.
hotel_df = ideal_weather_condition.reset_index(drop=True)
#append 2 new columns
#"Hotel Name and Hotel Address column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Address"] = ""
#* Plot the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**.

In [6]:
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address
0,jining,35.41,116.58,78.17,59,0,3.22,CN,1595203352,,
1,yongan,39.7,113.69,72.75,42,0,4.74,CN,1595203353,,
2,cherskiy,68.75,161.3,72.34,40,0,4.81,RU,1595203176,,
3,gonbad-e qabus,37.25,55.17,77.0,73,0,2.24,IR,1595203327,,
4,plomarion,38.98,26.37,73.4,78,0,4.7,GR,1595203376,,


In [7]:
# setup google api
text_serch_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
#params initial 
params1 = {
        "query" : "Hotel",
        "keyword": "hotel",
        "radius": 5000, # 5000 meters
        "types": "lodging",
        "key": gkey
    } 

# Loop all citys 
for index, row in hotel_df.iterrows():
    location = "{0},{1}".format( row["Lat"], row["Lng"])
    #obtain the location
    params1["location"] = location

    try:
        #query api
        response = requests.get(text_serch_url, params=params1).json()
        #get hotel name
        hotel_df.loc[index, "Hotel Name"]= response['results'][0]["name"]    
        #get real address
        hotel_df.loc[index, "Hotel Address"]= response['results'][0]["formatted_address"]
        #convert full country name US->USA, CA->Canada
        hotel_df.loc[index, "Country"] = pycountry.countries.get(alpha_2= hotel_df.loc[index, "Country"]).name
    except:
        hotel_df.loc[index, "Hotel Name"] = "hotel not found"
        


In [8]:
hotel_df = hotel_df.dropna()
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address
0,jining,35.41,116.58,78.17,59,0,3.22,China,1595203352,Wanda Realm Jining,"China, Shandong, Jining, 市中区太白楼东路59号 邮政编码: 272000"
1,yongan,39.7,113.69,72.75,42,0,4.74,China,1595203353,Yijintang Inn,"Yide Street, Hengshan S Rd, Hunyuan County, Da..."
2,cherskiy,68.75,161.3,72.34,40,0,4.81,Russian Federation,1595203176,Best Western Capistrano Inn,"27174 Ortega Hwy, San Juan Capistrano, CA 9267..."
3,gonbad-e qabus,37.25,55.17,77.0,73,0,2.24,"Iran, Islamic Republic of",1595203327,Hotel,"Golestan Province, گنبد كاووس،، Iran"
4,plomarion,38.98,26.37,73.4,78,0,4.7,Greece,1595203376,A-LUXURY VILLAS,"1, Gimnastiriou, PLOMARI LESVOS GREECE 812 00,..."
5,abrau-dyurso,44.69,37.59,75.2,60,0,6.71,Russian Federation,1595203378,"Guest house ""Melissa""","улица Южная, 18, Yuzhnaya Ozereyevka, Krasnoda..."
6,binzhou,37.37,118.02,78.01,71,0,7.2,China,1595203382,Binzhou Hotel,"762 Bohai 18th Rd, Bincheng District, Binzhou,..."
7,abu samrah,35.3,37.18,70.95,44,0,1.86,Syrian Arab Republic,1595203251,Borj hama,"Hama, Syria"
8,nurota,40.56,65.69,77.0,23,0,4.7,Uzbekistan,1595203391,Guest House Ruslan Nurata & Tours,"St. Okhunboboyev, h. 2 Nurata UZ, 210700, Uzbe..."
9,anzio,41.49,12.62,72.0,78,0,1.01,Italy,1595203396,Astura Palace Hotel,"V.le Giacomo Matteotti, 75, 00048 Nettuno RM, ..."


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name </dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Address</dt><dd>{Hotel Address}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[['Lat', 'Lng']].astype(float)

marker_layer = gmaps.marker_layer( locations,info_box_content = hotel_info)
fig = gmaps.figure(center=(-3.460047, 22.445250), zoom_level=2)
fig.add_layer(marker_layer)

locations = city_weather_df[['Lat', 'Lng']].astype(float)
temp_rate = city_weather_df["Max Temp"].astype(float)
#setup the center of map
gmap_layer = gmaps.heatmap_layer(
                                locations, 
                                weights = temp_rate,
                                dissipating=False,
                                max_intensity=100,
                                point_radius = 2)
fig.add_layer(gmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))